In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
from torch.utils.data import DataLoader, TensorDataset
import optuna

Загрузка данных и создание датасета

In [ ]:
df = pd.read_csv('daily_accidents.csv', parse_dates=['CRASH DATE'])
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
df.set_index('CRASH DATE', inplace=True)

df = df[['ACCIDENT_COUNT']]

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
df_scaled = scaler.fit_transform(df)

In [ ]:
X = []
y = []

seq_length = 30

for i in range(len(df_scaled) - seq_length):
    X.append(df_scaled[i:i + seq_length])
    y.append(df_scaled[i + seq_length])

In [ ]:
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512)

Модель

In [ ]:
class LSTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_heads, output_size):
        super(LSTransformer, self).__init__()

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=num_layers)

        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        transformer_out = self.transformer_encoder(lstm_out)

        output = self.fc(transformer_out[:, -1, :])
        return output

Функция подбора гиперпараметров

In [ ]:
def objective_lstransformer(trial):
    num_heads = trial.suggest_categorical("num_heads", [2, 4, 8])

    hidden_size_options = [hs for hs in range(32, 257, 8) if hs % num_heads == 0]
    hidden_size = trial.suggest_categorical("hidden_size", hidden_size_options)

    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    model = LSTransformer(
        input_size=X_train.shape[2],
        hidden_size=hidden_size,
        num_layers=num_layers,
        num_heads=num_heads,
        output_size=1
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    model.train()
    for epoch in range(20):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

    return test_loss / len(test_loader)


Подбор гиперпараметров

In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_lstransformer, n_trials=30)

print("Best parameters:", study.best_params)

[I 2025-05-05 14:35:40,248] A new study created in memory with name: no-name-48b9ed06-caff-4c39-be6e-ec9b88fa7bb6
[I 2025-05-05 14:49:39,449] Trial 0 finished with value: 0.005050899228081107 and parameters: {'num_heads': 2, 'hidden_size': 88, 'num_layers': 3, 'lr': 0.0003707330501322261}. Best is trial 0 with value: 0.005050899228081107.
[I 2025-05-05 15:21:31,481] Trial 1 finished with value: 0.2530614659190178 and parameters: {'num_heads': 8, 'hidden_size': 248, 'num_layers': 3, 'lr': 0.0006380949703953669}. Best is trial 0 with value: 0.005050899228081107.
[I 2025-05-05 15:40:16,503] Trial 2 finished with value: 0.24274666607379913 and parameters: {'num_heads': 8, 'hidden_size': 232, 'num_layers': 2, 'lr': 0.0022387685111331574}. Best is trial 0 with value: 0.005050899228081107.
[I 2025-05-05 15:48:15,270] Trial 3 finished with value: 0.0163142466917634 and parameters: {'num_heads': 8, 'hidden_size': 192, 'num_layers': 1, 'lr': 0.005237673075658886}. Best is trial 0 with value: 0.0

Best parameters: {'num_heads': 4, 'hidden_size': 168, 'num_layers': 1, 'lr': 0.00020864254903143865}


Параметры модели и инициализация

In [ ]:
input_size = 1
hidden_size = 168
num_layers = 1
num_heads = 4
output_size = 1
learning_rate = 0.00020864254903143865
epochs = 100
batch_size = 256

In [22]:
model = LSTransformer(input_size, hidden_size, num_layers, num_heads, output_size)

In [24]:
criterion = nn.MSELoss()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Обучение

In [ ]:
start_time = time.time()
train_loss = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    batch_count = 0

    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]


        optimizer.zero_grad()

        output = model(X_batch)

        if y_batch.ndim == 1:
            y_batch = y_batch.unsqueeze(1)

        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        batch_count += 1

    avg_loss = running_loss / batch_count
    train_loss.append(avg_loss)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}')

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

Epoch 10/100, Loss: 0.0407
Epoch 20/100, Loss: 0.0278
Epoch 30/100, Loss: 0.0262
Epoch 40/100, Loss: 0.0322
Epoch 50/100, Loss: 0.0331
Epoch 60/100, Loss: 0.0367
Epoch 70/100, Loss: 0.0268
Epoch 80/100, Loss: 0.0273
Epoch 90/100, Loss: 0.0270
Epoch 100/100, Loss: 0.0277
Training completed in 1983.72 seconds


Предсказание и метрики

In [ ]:
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train).detach().numpy()
    y_pred_test = model(X_test).detach().numpy()

In [ ]:
y_pred_train = scaler.inverse_transform(y_pred_train)
y_pred_test = scaler.inverse_transform(y_pred_test)
y_train = scaler.inverse_transform(y_train.numpy())
y_test = scaler.inverse_transform(y_test.numpy())

In [ ]:
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

correlation_train = np.corrcoef(y_train.flatten(), y_pred_train.flatten())[0, 1]
correlation_test = np.corrcoef(y_test.flatten(), y_pred_test.flatten())[0, 1]

print(f'Train RMSE: {rmse_train:.4f}, MAE: {mae_train:.4f}, R²: {r2_train:.4f}, Correlation: {correlation_train:.4f}')
print(f'Test RMSE: {rmse_test:.4f}, MAE: {mae_test:.4f}, R²: {r2_test:.4f}, Correlation: {correlation_test:.4f}')

Train RMSE: 95.2911, MAE: 73.6515, R²: 0.6511, Correlation: 0.8657
Test RMSE: 30.5338, MAE: 24.0883, R²: 0.2525, Correlation: 0.5309
